# PCA e Redução de Dimensionalidade

## Objetivos

- Compreender o problema da alta dimensionalidade
- Implementar e aplicar PCA (Principal Component Analysis)
- Interpretar componentes principais
- Aplicar PCA para visualização e pré-processamento
- Comparar com outras técnicas de redução de dimensionalidade

## Pré-requisitos

- Álgebra linear básica (vetores, matrizes)
- Conceitos de variância e covariância
- Normalização de dados


## 1. O Problema da Alta Dimensionalidade

### Curse of Dimensionality:

- **Esparsidade**: Pontos ficam muito distantes em alta dimensão
- **Overfitting**: Modelos se tornam complexos demais
- **Visualização**: Impossível visualizar dados com >3 dimensões
- **Computação**: Algoritmos ficam lentos

### Soluções:

- **Feature Selection**: Escolher features mais importantes
- **Feature Extraction**: Criar novas features (PCA, t-SNE)
- **Regularização**: Penalizar complexidade do modelo


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits, load_breast_cancer, make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.manifold import TSNE
import warnings

warnings.filterwarnings("ignore")

# Configurar visualização
plt.style.use("default")
sns.set_palette("husl")
np.random.seed(42)

## 2. Demonstrando a Curse of Dimensionality


In [ ]:
# Demonstrar como distâncias mudam com dimensionalidade
def analyze_distances_by_dimension(n_samples=1000, max_dim=50):
    dimensions = range(1, max_dim + 1, 5)
    mean_distances = []
    std_distances = []

    for dim in dimensions:
        # Gerar pontos aleatórios
        points = np.random.normal(0, 1, (n_samples, dim))

        # Calcular distâncias do primeiro ponto para todos os outros
        distances = np.sqrt(np.sum((points[0] - points[1:]) ** 2, axis=1))

        mean_distances.append(np.mean(distances))
        std_distances.append(np.std(distances))

    return dimensions, mean_distances, std_distances


# Analisar distâncias
dims, means, stds = analyze_distances_by_dimension()

# Plotar resultados
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(dims, means, "b-o", label="Distância Média")
plt.fill_between(dims, np.array(means) - np.array(stds), np.array(means) + np.array(stds), alpha=0.3)
plt.xlabel("Dimensionalidade")
plt.ylabel("Distância Euclidiana")
plt.title("Distância Média entre Pontos Aleatórios")
plt.grid(True, alpha=0.3)
plt.legend()

# Razão entre distância máxima e mínima
plt.subplot(1, 2, 2)
relative_variation = np.array(stds) / np.array(means)
plt.plot(dims, relative_variation, "r-o")
plt.xlabel("Dimensionalidade")
plt.ylabel("Variação Relativa (std/mean)")
plt.title("Variabilidade Relativa das Distâncias")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observações sobre Curse of Dimensionality:")
print(f"- Em alta dimensão, distâncias tendem a convergir")
print(f"- Variação relativa diminui: {relative_variation[0]:.3f} (2D) -> {relative_variation[-1]:.3f} ({dims[-1]}D)")
print(f"- Conceito de 'vizinhança próxima' perde significado")

## 3. Principal Component Analysis (PCA)

### Ideia Central:

- Encontrar **direções de máxima variância** nos dados
- **Projetar dados** nessas direções (componentes principais)
- **Reduzir dimensionalidade** mantendo informação importante

### Algoritmo:

1. **Centralizar dados** (média = 0)
2. **Calcular matriz de covariância**
3. **Encontrar autovalores e autovetores**
4. **Ordenar por autovalor** (maior variância primeiro)
5. **Projetar dados** nos autovetores escolhidos


In [ ]:
# Implementação manual do PCA (para compreensão)
class PCAManual:
    def __init__(self, n_components=2):
        self.n_components = n_components

    def fit(self, X):
        # 1. Centralizar dados
        self.mean_ = np.mean(X, axis=0)
        X_centered = X - self.mean_

        # 2. Calcular matriz de covariância
        cov_matrix = np.cov(X_centered.T)

        # 3. Calcular autovalores e autovetores
        eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

        # 4. Ordenar por autovalor (maior primeiro)
        idx = np.argsort(eigenvalues)[::-1]
        self.eigenvalues_ = eigenvalues[idx]
        self.eigenvectors_ = eigenvectors[:, idx]

        # 5. Selecionar componentes principais
        self.components_ = self.eigenvectors_[:, : self.n_components].T

        # Calcular variância explicada
        total_variance = np.sum(self.eigenvalues_)
        self.explained_variance_ratio_ = self.eigenvalues_[: self.n_components] / total_variance

        return self

    def transform(self, X):
        # Centralizar e projetar
        X_centered = X - self.mean_
        return X_centered @ self.components_.T

    def fit_transform(self, X):
        return self.fit(X).transform(X)

    def inverse_transform(self, X_pca):
        # Reconstruir dados originais (aproximação)
        return X_pca @ self.components_ + self.mean_


# Criar dados 2D simples para visualização
np.random.seed(42)
n_samples = 200
X_simple = np.random.multivariate_normal(mean=[0, 0], cov=[[3, 1.5], [1.5, 1]], size=n_samples)

# Aplicar PCA manual
pca_manual = PCAManual(n_components=2)
X_pca_manual = pca_manual.fit_transform(X_simple)

# Comparar com sklearn
pca_sklearn = PCA(n_components=2)
X_pca_sklearn = pca_sklearn.fit_transform(X_simple)

# Visualizar
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Dados originais com componentes principais
axes[0].scatter(X_simple[:, 0], X_simple[:, 1], alpha=0.7)
axes[0].set_xlabel("Feature 1")
axes[0].set_ylabel("Feature 2")
axes[0].set_title("Dados Originais")
axes[0].grid(True, alpha=0.3)
axes[0].axis("equal")

# Adicionar vetores dos componentes principais
origin = pca_manual.mean_
for i, (component, variance) in enumerate(zip(pca_manual.components_, pca_manual.explained_variance_ratio_)):
    axes[0].arrow(
        origin[0],
        origin[1],
        component[0] * 3,
        component[1] * 3,
        head_width=0.1,
        head_length=0.1,
        fc=f"C{i}",
        ec=f"C{i}",
        label=f"PC{i+1} ({variance:.1%} var)",
    )
axes[0].legend()

# Dados transformados (manual)
axes[1].scatter(X_pca_manual[:, 0], X_pca_manual[:, 1], alpha=0.7)
axes[1].set_xlabel("PC1")
axes[1].set_ylabel("PC2")
axes[1].set_title("PCA Manual")
axes[1].grid(True, alpha=0.3)
axes[1].axis("equal")

# Dados transformados (sklearn)
axes[2].scatter(X_pca_sklearn[:, 0], X_pca_sklearn[:, 1], alpha=0.7)
axes[2].set_xlabel("PC1")
axes[2].set_ylabel("PC2")
axes[2].set_title("PCA Sklearn")
axes[2].grid(True, alpha=0.3)
axes[2].axis("equal")

plt.tight_layout()
plt.show()

print(f"Variância explicada (manual): {pca_manual.explained_variance_ratio_}")
print(f"Variância explicada (sklearn): {pca_sklearn.explained_variance_ratio_}")
print(f"Diferença máxima: {np.max(np.abs(np.abs(X_pca_manual) - np.abs(X_pca_sklearn))):.10f}")

## 4. PCA em Dataset Real: Dígitos Manuscritos


In [ ]:
# Carregar dataset de dígitos
digits = load_digits()
X_digits, y_digits = digits.data, digits.target

print(f"Dataset de dígitos:")
print(f"Shape: {X_digits.shape}")
print(f"Classes: {np.unique(y_digits)}")
print(f"Cada imagem: 8x8 pixels = 64 features")

# Visualizar algumas imagens
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    ax.imshow(digits.images[i], cmap="gray")
    ax.set_title(f"Dígito: {y_digits[i]}")
    ax.axis("off")

plt.suptitle("Exemplos do Dataset de Dígitos", fontsize=14)
plt.tight_layout()
plt.show()

# Normalizar dados
scaler = StandardScaler()
X_digits_scaled = scaler.fit_transform(X_digits)

# Aplicar PCA com diferentes números de componentes
n_components_list = [2, 5, 10, 20, 30, 40, 50, 60]
explained_variances = []

for n_comp in n_components_list:
    pca = PCA(n_components=n_comp)
    pca.fit(X_digits_scaled)
    explained_variances.append(np.sum(pca.explained_variance_ratio_))

# Plotar variância explicada acumulada
plt.figure(figsize=(10, 6))
plt.plot(n_components_list, explained_variances, "bo-", linewidth=2, markersize=8)
plt.xlabel("Número de Componentes")
plt.ylabel("Variância Explicada Acumulada")
plt.title("Variância Explicada vs Número de Componentes")
plt.grid(True, alpha=0.3)

# Adicionar linhas de referência
plt.axhline(y=0.8, color="red", linestyle="--", alpha=0.7, label="80% variância")
plt.axhline(y=0.9, color="orange", linestyle="--", alpha=0.7, label="90% variância")
plt.axhline(y=0.95, color="green", linestyle="--", alpha=0.7, label="95% variância")
plt.legend()

# Anotar pontos importantes
for i, (n_comp, var_exp) in enumerate(zip(n_components_list, explained_variances)):
    if var_exp >= 0.8 and explained_variances[i - 1] < 0.8:
        plt.annotate(
            f"{n_comp} comp.\n{var_exp:.1%}",
            xy=(n_comp, var_exp),
            xytext=(n_comp + 10, var_exp - 0.05),
            arrowprops=dict(arrowstyle="->", color="red"),
        )
        break

plt.tight_layout()
plt.show()

print("\nVariância explicada por número de componentes:")
for n_comp, var_exp in zip(n_components_list, explained_variances):
    print(f"{n_comp:2d} componentes: {var_exp:.1%}")

## 5. Visualização com PCA


In [ ]:
# Aplicar PCA para visualização (2D e 3D)
pca_2d = PCA(n_components=2)
X_digits_2d = pca_2d.fit_transform(X_digits_scaled)

pca_3d = PCA(n_components=3)
X_digits_3d = pca_3d.fit_transform(X_digits_scaled)

# Visualização 2D
plt.figure(figsize=(15, 5))

# PCA 2D - colorido por classe
plt.subplot(1, 3, 1)
scatter = plt.scatter(X_digits_2d[:, 0], X_digits_2d[:, 1], c=y_digits, cmap="tab10", alpha=0.7, s=20)
plt.xlabel(f"PC1 ({pca_2d.explained_variance_ratio_[0]:.1%} variância)")
plt.ylabel(f"PC2 ({pca_2d.explained_variance_ratio_[1]:.1%} variância)")
plt.title("PCA 2D - Dígitos")
plt.colorbar(scatter, label="Dígito")
plt.grid(True, alpha=0.3)

# Destacar alguns dígitos específicos
plt.subplot(1, 3, 2)
for digit in [0, 1, 4, 7]:
    mask = y_digits == digit
    plt.scatter(X_digits_2d[mask, 0], X_digits_2d[mask, 1], label=f"Dígito {digit}", alpha=0.7, s=20)
plt.xlabel(f"PC1")
plt.ylabel(f"PC2")
plt.title("PCA 2D - Dígitos Selecionados")
plt.legend()
plt.grid(True, alpha=0.3)

# Densidade por região
plt.subplot(1, 3, 3)
plt.hexbin(X_digits_2d[:, 0], X_digits_2d[:, 1], gridsize=30, cmap="Blues")
plt.xlabel(f"PC1")
plt.ylabel(f"PC2")
plt.title("Densidade de Pontos")
plt.colorbar(label="Contagem")

plt.tight_layout()
plt.show()

print(f"PCA 2D - Variância total explicada: {np.sum(pca_2d.explained_variance_ratio_):.1%}")
print(f"PCA 3D - Variância total explicada: {np.sum(pca_3d.explained_variance_ratio_):.1%}")

## 6. Reconstrução de Imagens com PCA


In [ ]:
# Reconstruir imagens com diferentes números de componentes
def reconstruct_images(X, n_components_list, image_idx=0):
    """Reconstrói uma imagem usando diferentes números de componentes PCA"""
    original_image = X[image_idx].reshape(8, 8)
    reconstructions = []

    for n_comp in n_components_list:
        pca = PCA(n_components=n_comp)
        X_pca = pca.fit_transform(X)
        X_reconstructed = pca.inverse_transform(X_pca)

        reconstructed_image = X_reconstructed[image_idx].reshape(8, 8)
        reconstructions.append(reconstructed_image)

    return original_image, reconstructions


# Testar reconstrução
test_image_idx = 0
n_components_test = [1, 2, 5, 10, 20, 40]

original, reconstructed_list = reconstruct_images(X_digits_scaled, n_components_test, test_image_idx)

# Visualizar reconstruções
fig, axes = plt.subplots(2, len(n_components_test) + 1, figsize=(18, 6))

# Linha superior: reconstruções
axes[0, 0].imshow(digits.images[test_image_idx], cmap="gray")
axes[0, 0].set_title(f"Original\nDígito {y_digits[test_image_idx]}")
axes[0, 0].axis("off")

for i, (n_comp, reconstructed) in enumerate(zip(n_components_test, reconstructed_list)):
    axes[0, i + 1].imshow(reconstructed, cmap="gray")
    axes[0, i + 1].set_title(f"{n_comp} comp.")
    axes[0, i + 1].axis("off")

# Linha inferior: erro de reconstrução
axes[1, 0].text(0.5, 0.5, "Erro de\nReconstrução", ha="center", va="center", transform=axes[1, 0].transAxes)
axes[1, 0].axis("off")

for i, (n_comp, reconstructed) in enumerate(zip(n_components_test, reconstructed_list)):
    error = np.abs(digits.images[test_image_idx] - reconstructed)
    im = axes[1, i + 1].imshow(error, cmap="Reds")
    axes[1, i + 1].set_title(f"MSE: {np.mean(error**2):.3f}")
    axes[1, i + 1].axis("off")

plt.tight_layout()
plt.show()

# Calcular MSE para diferentes números de componentes
mse_scores = []
for n_comp in range(1, 65):
    pca = PCA(n_components=n_comp)
    X_pca = pca.fit_transform(X_digits_scaled)
    X_reconstructed = pca.inverse_transform(X_pca)
    mse = np.mean((X_digits_scaled - X_reconstructed) ** 2)
    mse_scores.append(mse)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 65), mse_scores, "b-", linewidth=2)
plt.xlabel("Número de Componentes")
plt.ylabel("MSE de Reconstrução")
plt.title("Erro de Reconstrução vs Número de Componentes")
plt.grid(True, alpha=0.3)
plt.yscale("log")
plt.show()

print(f"\nTrade-off entre compressão e qualidade:")
for n_comp in [5, 10, 20, 30]:
    compression_ratio = n_comp / 64
    mse = mse_scores[n_comp - 1]
    print(f"{n_comp:2d} componentes: {compression_ratio:.1%} dos dados originais, MSE = {mse:.4f}")

## 7. PCA como Pré-processamento para ML


In [ ]:
# Comparar performance de ML com e sem PCA
def compare_ml_with_pca(X, y, n_components_list, test_size=0.2):
    """
    Compara performance de modelos ML com diferentes números de componentes PCA
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    results = []

    # Modelo sem PCA
    rf_baseline = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_baseline.fit(X_train, y_train)
    baseline_score = rf_baseline.score(X_test, y_test)

    results.append(
        {
            "n_components": "Original (64)",
            "accuracy": baseline_score,
            "n_features": X.shape[1],
            "compression_ratio": 1.0,
        }
    )

    # Modelos com PCA
    for n_comp in n_components_list:
        # Aplicar PCA
        pca = PCA(n_components=n_comp)
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        # Treinar modelo
        rf_pca = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_pca.fit(X_train_pca, y_train)
        pca_score = rf_pca.score(X_test_pca, y_test)

        results.append(
            {
                "n_components": n_comp,
                "accuracy": pca_score,
                "n_features": n_comp,
                "compression_ratio": n_comp / X.shape[1],
            }
        )

    return pd.DataFrame(results)


# Testar com dígitos
n_components_ml = [2, 5, 10, 15, 20, 30, 40, 50]
results_df = compare_ml_with_pca(X_digits_scaled, y_digits, n_components_ml)

print("Performance de Random Forest com PCA:")
print("=" * 50)
print(results_df.round(3))

# Visualizar resultados
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy vs número de componentes
numeric_results = results_df[results_df["n_components"] != "Original (64)"]
baseline_acc = results_df[results_df["n_components"] == "Original (64)"]["accuracy"].iloc[0]

axes[0].plot(
    numeric_results["n_components"], numeric_results["accuracy"], "bo-", linewidth=2, markersize=8, label="Com PCA"
)
axes[0].axhline(y=baseline_acc, color="red", linestyle="--", label=f"Sem PCA ({baseline_acc:.3f})")
axes[0].set_xlabel("Número de Componentes PCA")
axes[0].set_ylabel("Accuracy")
axes[0].set_title("Performance vs Redução de Dimensionalidade")
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Trade-off: accuracy vs compressão
axes[1].scatter(
    numeric_results["compression_ratio"],
    numeric_results["accuracy"],
    s=100,
    alpha=0.7,
    c=numeric_results["n_components"],
    cmap="viridis",
)
axes[1].axhline(y=baseline_acc, color="red", linestyle="--", alpha=0.7)
axes[1].axvline(x=1.0, color="red", linestyle="--", alpha=0.7)
axes[1].set_xlabel("Taxa de Compressão (n_comp / 64)")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Trade-off: Accuracy vs Compressão")
axes[1].grid(True, alpha=0.3)

# Adicionar colorbar
colorbar = plt.colorbar(axes[1].collections[0], ax=axes[1])
colorbar.set_label("Número de Componentes")

# Anotar pontos interessantes
best_ratio_idx = numeric_results["accuracy"].idxmax()
best_comp = numeric_results.loc[best_ratio_idx, "n_components"]
best_acc = numeric_results.loc[best_ratio_idx, "accuracy"]
best_ratio = numeric_results.loc[best_ratio_idx, "compression_ratio"]

axes[1].annotate(
    f"Melhor: {best_comp} comp.\nAcc: {best_acc:.3f}",
    xy=(best_ratio, best_acc),
    xytext=(best_ratio - 0.2, best_acc + 0.02),
    arrowprops=dict(arrowstyle="->", color="red"),
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
)

plt.tight_layout()
plt.show()

print(f"\nMelhor configuração: {best_comp} componentes")
print(f"Accuracy: {best_acc:.3f} vs {baseline_acc:.3f} (baseline)")
print(f"Redução de features: {64} -> {best_comp} ({best_ratio:.1%})")
print(f"Diferença de performance: {best_acc - baseline_acc:+.3f}")

## 8. Interpretando Componentes Principais


In [ ]:
# Visualizar os primeiros componentes principais como imagens
pca_interpretation = PCA(n_components=16)
pca_interpretation.fit(X_digits_scaled)

# Plotar componentes principais
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.flatten()

for i in range(16):
    component_image = pca_interpretation.components_[i].reshape(8, 8)

    # Plotar componente
    im = axes[i].imshow(component_image, cmap="RdBu_r")
    axes[i].set_title(f"PC{i+1}\n({pca_interpretation.explained_variance_ratio_[i]:.1%} var)")
    axes[i].axis("off")

    # Adicionar colorbar para alguns componentes
    if i in [0, 1, 2, 3]:
        plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

plt.suptitle("Primeiros 16 Componentes Principais\n(Padrões mais importantes nos dados)", fontsize=14)
plt.tight_layout()
plt.show()

# Analisar contribuição de cada pixel
print("Interpretação dos Componentes Principais:")
print("=" * 50)
print("PC1: Captura contraste geral (fundo vs dígito)")
print("PC2-PC4: Capturam formas e orientações específicas")
print("Componentes seguintes: Detalhes mais sutis e ruído")

# Mostrar como diferentes dígitos se projetam nos primeiros PCs
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for digit in range(10):
    digit_mask = y_digits == digit
    digit_projections = X_digits_2d[digit_mask]

    axes[digit].scatter(digit_projections[:, 0], digit_projections[:, 1], alpha=0.6, s=20)
    axes[digit].set_title(f"Dígito {digit}")
    axes[digit].set_xlabel("PC1")
    axes[digit].set_ylabel("PC2")
    axes[digit].grid(True, alpha=0.3)

plt.suptitle("Projeção de cada dígito nos primeiros 2 PCs", fontsize=14)
plt.tight_layout()
plt.show()

## 9. Comparação com t-SNE


In [ ]:
# Comparar PCA com t-SNE para visualização
# Usar subset dos dados para acelerar t-SNE
n_samples_tsne = 500
indices = np.random.choice(len(X_digits), n_samples_tsne, replace=False)
X_subset = X_digits_scaled[indices]
y_subset = y_digits[indices]

# PCA 2D
pca_viz = PCA(n_components=2)
X_pca_viz = pca_viz.fit_transform(X_subset)

# t-SNE 2D
print("Aplicando t-SNE (pode demorar alguns segundos...)")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_subset)

# Visualizar comparação
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# PCA
scatter1 = axes[0].scatter(X_pca_viz[:, 0], X_pca_viz[:, 1], c=y_subset, cmap="tab10", alpha=0.7, s=30)
axes[0].set_xlabel(f"PC1 ({pca_viz.explained_variance_ratio_[0]:.1%})")
axes[0].set_ylabel(f"PC2 ({pca_viz.explained_variance_ratio_[1]:.1%})")
axes[0].set_title(f"PCA 2D\n(Variância total: {np.sum(pca_viz.explained_variance_ratio_):.1%})")
axes[0].grid(True, alpha=0.3)

# t-SNE
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_subset, cmap="tab10", alpha=0.7, s=30)
axes[1].set_xlabel("t-SNE 1")
axes[1].set_ylabel("t-SNE 2")
axes[1].set_title("t-SNE 2D\n(Preserva estrutura local)")
axes[1].grid(True, alpha=0.3)

# Adicionar colorbar
plt.colorbar(scatter1, ax=axes[0], label="Dígito")
plt.colorbar(scatter2, ax=axes[1], label="Dígito")

plt.tight_layout()
plt.show()

# Calcular métricas de separação
from sklearn.metrics import silhouette_score

pca_silhouette = silhouette_score(X_pca_viz, y_subset)
tsne_silhouette = silhouette_score(X_tsne, y_subset)

print("\nComparação PCA vs t-SNE:")
print("=" * 30)
print(f"PCA Silhouette Score:  {pca_silhouette:.3f}")
print(f"t-SNE Silhouette Score: {tsne_silhouette:.3f}")
print("\nCaracterísticas:")
print("PCA:")
print("  + Rápido e determinístico")
print("  + Preserva variância global")
print("  + Permite reconstrução")
print("  - Assume relações lineares")
print("\nt-SNE:")
print("  + Preserva estrutura local")
print("  + Melhor separação visual")
print("  - Lento para dados grandes")
print("  - Não determinístico")
print("  - Apenas para visualização")

## 10. Resumo e Boas Práticas

### Quando usar PCA:

- **Redução de dimensionalidade** para acelerar algoritmos
- **Visualização** de dados multidimensionais
- **Compressão** com perda controlada
- **Pré-processamento** para evitar curse of dimensionality
- **Detecção de outliers** (pontos com alta reconstrução de erro)

### Limitações do PCA:

- Assume **relações lineares**
- Sensível à **escala das features** (sempre normalizar)
- Componentes podem ser **difíceis de interpretar**
- Pode **remover informação importante** para classificação

### Processo recomendado:

1. **Normalizar dados** (StandardScaler)
2. **Analisar variância explicada** para escolher número de componentes
3. **Validar performance** de ML com diferentes números de componentes
4. **Interpretar componentes** quando possível
5. **Considerar alternativas** (t-SNE para visualização, kernel PCA para não-linearidade)

### Regras práticas:

- **85-95% da variância** geralmente suficiente
- **Menos componentes que amostras** (n_components < n_samples)
- **Testar com validação cruzada** para escolher número ótimo
- **Aplicar PCA apenas no conjunto de treino**, depois transformar teste
